Objectifs du notebook :

- Nettoyer les données pour obtenir un seul fichier contenant l'échantillon qui sera utilisée.
- Vérifier la présence de valeurs manquantes ou erronées.
- S'assurer de réunir uniquement les données nécessaires pour la base de données.
- Vérifier le typage des données correspondant à la base de données prévue.

[Lien DrawDB](https://drawdb.vercel.app/editor?shareId=e6c18b8ae53063fa1dfa9cc8a849605f)

In [2]:
import matplotlib.pyplot as plt
from rich import print_json, print
from tqdm import tqdm
import pandas as pd
import glob
import os

Récupération de la liste des fichiers du dataset disponible

In [ ]:
def get_csv_list(part_number):
    if type(part_number) == list:
        for part in part_number:
            csv_files = glob.glob(f"./data/part_{part}/*.csv")
    else:
        csv_files = glob.glob(f"./data/part_{part_number}/*.csv")
    return csv_files

[
    './data/part_1/may_july_chunk_16.csv',
    './data/part_1/may_july_chunk_13.csv',
    './data/part_1/may_july_chunk_1.csv',
    './data/part_1/may_july_chunk_17.csv',
    './data/part_1/may_july_chunk_3.csv',
    './data/part_1/may_july_chunk_20.csv',
    './data/part_1/may_july_chunk_15.csv',
    './data/part_1/may_july_chunk_7.csv',
    './data/part_1/may_july_chunk_5.csv',
    './data/part_1/may_july_chunk_6.csv',
    './data/part_1/may_july_chunk_4.csv',
    './data/part_1/may_july_chunk_14.csv',
    './data/part_1/may_july_chunk_10.csv',
    './data/part_1/may_july_chunk_8.csv',
    './data/part_1/may_july_chunk_2.csv',
    './data/part_1/may_july_chunk_9.csv',
    './data/part_1/may_july_chunk_19.csv',
    './data/part_1/may_july_chunk_18.csv',
    './data/part_1/may_july_chunk_12.csv',
    './data/part_1/may_july_chunk_11.csv'
]

Création du DataFrame final

In [6]:
x_post_df = pd.DataFrame({
    "user_id": pd.Series(dtype="int"),
    "lang": pd.Series(dtype="str"),
    "text": pd.Series(dtype="str"),
    "date": pd.Series(dtype="datetime64[ns]"),
    "like_count": pd.Series(dtype="int"),
    "reply_count": pd.Series(dtype="int"),
    "retweet_count": pd.Series(dtype="int"),
    "quote_count": pd.Series(dtype="int"),
})

print(x_post_df.dtypes)
x_post_df.to_parquet("x_post.parquet", index=False)

user_id                   int64
lang                     object
text                     object
date             datetime64[ns]
like_count                int64
reply_count               int64
retweet_count             int64
quote_count               int64
dtype: object

Clean function
- input : chemin du csv a ajouter, le dataframe original
- output : le dataframe avec les nouvelles lignes

In [ ]:
def clean_file(path, x_post_df):
    current_df = pd.read_csv(path)
    for i, row in tqdm(current_df.iterrows(), total=len(current_df), desc=f"Processing {os.path.basename(path)}"):
        try:
            # Vérification des données
            # Nous voulons :
            # - Les lignes sans valeurs nulles
            # - Les tweets en français
            # - Sans retweets ni citation de tweets.
            required_columns = ["id", "lang", "text", "date", "likeCount", "replyCount", "retweetCount", "quoteCount"]
            if any(col not in row or pd.isna(row[col]) for col in required_columns):
                continue
            if row["lang"] != "fr": # Choisir language voulu
                continue
            if row["quotedTweet"] == True:
                continue
            if row["retweetedTweet"] == True:
                continue
        
            user_id = int(row["id"])
            lang = row["lang"]
            text = row["text"]
            date = row["date"]
            like_count = row["likeCount"]
            reply_count = row["replyCount"]
            retweet_count = row["retweetCount"]
            quote_count = row["quoteCount"]
            
            # Create a new row for the final dataframe
            new_row = pd.DataFrame({
                "user_id": [user_id],
                "lang": [lang],
                "text": [text],
                "date": [pd.to_datetime(date)],
                "like_count": [like_count],
                "reply_count": [reply_count],
                "retweet_count": [retweet_count],
                "quote_count": [quote_count]
            })
            
            # Append to the main dataframe
            x_post_df = pd.concat([x_post_df, new_row], ignore_index=True)
            
        except Exception as e:
            print(f"Error processing row {i} from file {os.path.basename(path)}: {str(e)}")
            continue
    
    return x_post_df
        

Cellule a utiliser pour actualiser x_post.parquet

In [ ]:
x_post_df = pd.read_parquet("x_post.parquet")

for path in get_csv_list(1): # Change the argument to choose the part number folder to treat
    print(f"Processing file: {path}")
    x_post_df = clean_file(path, x_post_df)

print("[bold yellow]x_post_df describe:[/bold yellow]")
print(x_post_df.describe())
print("[bold yellow]x_post_df head(5):[/bold yellow]")
print(x_post_df.head())
x_post_df.to_parquet("x_post.parquet", index=False)
print("[bold yellow]Sucessfuly saved to x_post.parquet[/bold yellow]")

Processing file: ./data/part_1/may_july_chunk_16.csv

Processing may_july_chunk_16.csv: 100%|██████████| 49984/49984 [00:02<00:00, 18378.70it/s]


Processing file: ./data/part_1/may_july_chunk_13.csv

Processing may_july_chunk_13.csv: 100%|██████████| 50000/50000 [00:02<00:00, 19438.63it/s]


Processing file: ./data/part_1/may_july_chunk_1.csv

...

x_post_df describe:

user_id                           date   like_count  reply_count  \
count  4.931000e+03                           4931  4931.000000  4931.000000   
mean   1.801254e+18  2024-06-12 23:07:08.554045696    12.640641     1.456702   
min    1.800679e+18            2024-06-12 00:00:00     0.000000     0.000000   
25%    1.800955e+18            2024-06-12 00:00:00     0.000000     0.000000   
50%    1.801280e+18            2024-06-13 00:00:00     0.000000     0.000000   
75%    1.801579e+18            2024-06-14 00:00:00     1.000000     1.000000   
max    1.801766e+18            2024-06-14 00:00:00  4702.000000   557.000000   
std    3.206305e+14                            NaN   138.843174    13.465933   

       retweet_count  quote_count  
count    4931.000000  4931.000000  
mean        4.275806     0.908335  
min         0.000000     0.000000  
25%         0.000000     0.000000  
50%         0.000000     0.000000  
75%         0.000000     0.000000  
max      1887.000000   604.000000  
std        54.479765    19.025810

x_post_df head(5):

user_id lang  \
0  1801761052310310976   fr   
1  1801760646603669666   fr   
2  1801760474058428922   fr   
3  1801760036869300601   fr   
4  1801759887405240335   fr   

                                                text       date  like_count  \
0  @benoitm_mtl le pape est pas vite vite ,les ga... 2024-06-14         0.0   
1                  @JdeMontreal Parlons de Biden !!! 2024-06-14         0.0   
2    @EscadronKoukou Des vrais partisans de Biden 😂😂 2024-06-14        12.0   
3  @leclerctaillage @rbrtcormier Qui tu propose ?... 2024-06-14         1.0   
4  @TThierryn @richardhetu Vrai ou pas que c’est ... 2024-06-14         0.0   

   reply_count  retweet_count  quote_count  
0          0.0            0.0          0.0  
1          0.0            0.0          0.0  
2          1.0            0.0          0.0  
3          1.0            0.0          0.0  
4          0.0            0.0          0.0

Sucessfuly saved to x_post.parquet